# 🎨 Stable Diffusion Backend Server for Frontend
### Google Colab + Cloudflared Tunnel
**Use with**: Your HTML/JS/CSS Frontend (Github Pages, Local, etc.)

This notebook provides a complete API backend for text-to-image, image-to-image, and inpainting.

## Step 1: Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch torchvision diffusers transformers accelerate safetensors flask flask-cors pyngrok pillow numpy xformers
print('✅ Dependencies installed!')

## Step 2: Install Cloudflared for HTTPS Tunnel

In [ ]:
# Install cloudflared
!wget -q https://github.com/cloudflare/wrangler/releases/download/wrangler-3.0.1/cloudflared-linux-amd64.deb -O /tmp/cloudflared.deb
!dpkg -i /tmp/cloudflared.deb > /dev/null 2>&1
print('✅ Cloudflared installed!')

## Step 3: Load Stable Diffusion Model

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionInpaintPipeline
from PIL import Image
import io
import base64
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import os

# Check for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'🖥️ Using device: {device}')

# Model ID (can be changed)
MODEL_ID = "runwayml/stable-diffusion-v1-5"
print(f'📦 Loading model: {MODEL_ID}...')

# Load pipelines
try:
    print('Loading txt2img pipeline...')
    pipe_txt2img = StableDiffusionPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        safety_checker=None
    ).to(device)
    pipe_txt2img.enable_xformers_memory_efficient_attention()
    print('✅ txt2img loaded')
    
    print('Loading img2img pipeline...')
    pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        safety_checker=None
    ).to(device)
    pipe_img2img.enable_xformers_memory_efficient_attention()
    print('✅ img2img loaded')
    
    print('Loading inpaint pipeline...')
    pipe_inpaint = StableDiffusionInpaintPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        safety_checker=None
    ).to(device)
    pipe_inpaint.enable_xformers_memory_efficient_attention()
    print('✅ inpaint loaded')
    
    print('\n✅ All pipelines ready!')
except Exception as e:
    print(f'❌ Error loading models: {e}')

## Step 4: Create Flask API Server

In [ ]:
# Create Flask app
app = Flask(__name__)
CORS(app)

# Global variables for progress tracking
generation_progress = {"current": 0, "total": 0}
is_generating = False

def image_to_base64(image):
    """Convert PIL Image to base64 string"""
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode()
    return f"data:image/png;base64,{img_str}"

def base64_to_image(base64_str):
    """Convert base64 string to PIL Image"""
    # Remove data URI prefix if present
    if ',' in base64_str:
        base64_str = base64_str.split(',')[1]
    
    image_data = base64.b64decode(base64_str)
    image = Image.open(io.BytesIO(image_data))
    return image.convert('RGB')

# API Endpoints

@app.route('/api/health', methods=['GET'])
def health():
    """Health check endpoint"""
    return jsonify({"status": "healthy", "message": "Server is running"}), 200

@app.route('/api/txt2img', methods=['POST'])
def txt2img():
    """Text to image generation"""
    global is_generating
    
    try:
        if is_generating:
            return jsonify({"error": "Generation already in progress"}), 429
        
        is_generating = True
        data = request.json
        
        # Get parameters
        prompt = data.get('prompt', '')
        negative_prompt = data.get('negative_prompt', '')
        steps = int(data.get('steps', 20))
        cfg_scale = float(data.get('cfg_scale', 7.5))
        width = int(data.get('width', 512))
        height = int(data.get('height', 512))
        seed = int(data.get('seed', -1))
        batch_size = int(data.get('batch_size', 1))
        
        # Set seed if provided
        if seed >= 0:
            generator = torch.Generator(device=device).manual_seed(seed)
        else:
            generator = None
        
        print(f'🎨 Generating txt2img: "{prompt}"')
        
        # Generate images
        result = pipe_txt2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=cfg_scale,
            height=height,
            width=width,
            num_images_per_prompt=batch_size,
            generator=generator
        )
        
        # Convert images to base64
        images = [image_to_base64(img) for img in result.images]
        
        is_generating = False
        return jsonify({"success": True, "images": images}), 200
    
    except Exception as e:
        is_generating = False
        print(f'❌ Error in txt2img: {e}')
        return jsonify({"error": str(e)}), 500

@app.route('/api/img2img', methods=['POST'])
def img2img():
    """Image to image generation"""
    global is_generating
    
    try:
        if is_generating:
            return jsonify({"error": "Generation already in progress"}), 429
        
        is_generating = True
        data = request.json
        
        # Get parameters
        prompt = data.get('prompt', '')
        negative_prompt = data.get('negative_prompt', '')
        image_base64 = data.get('image', '')
        steps = int(data.get('steps', 20))
        cfg_scale = float(data.get('cfg_scale', 7.5))
        strength = float(data.get('strength', 0.8))
        seed = int(data.get('seed', -1))
        batch_size = int(data.get('batch_size', 1))
        
        # Convert base64 to image
        image = base64_to_image(image_base64)
        
        # Set seed if provided
        if seed >= 0:
            generator = torch.Generator(device=device).manual_seed(seed)
        else:
            generator = None
        
        print(f'🎨 Generating img2img: "{prompt}" (strength: {strength})')
        
        # Generate images
        result = pipe_img2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=image,
            num_inference_steps=steps,
            guidance_scale=cfg_scale,
            strength=strength,
            num_images_per_prompt=batch_size,
            generator=generator
        )
        
        # Convert images to base64
        images = [image_to_base64(img) for img in result.images]
        
        is_generating = False
        return jsonify({"success": True, "images": images}), 200
    
    except Exception as e:
        is_generating = False
        print(f'❌ Error in img2img: {e}')
        return jsonify({"error": str(e)}), 500

@app.route('/api/inpaint', methods=['POST'])
def inpaint():
    """Inpainting generation"""
    global is_generating
    
    try:
        if is_generating:
            return jsonify({"error": "Generation already in progress"}), 429
        
        is_generating = True
        data = request.json
        
        # Get parameters
        prompt = data.get('prompt', '')
        negative_prompt = data.get('negative_prompt', '')
        image_base64 = data.get('image', '')
        mask_base64 = data.get('mask', '')
        steps = int(data.get('steps', 20))
        cfg_scale = float(data.get('cfg_scale', 7.5))
        strength = float(data.get('strength', 0.8))
        seed = int(data.get('seed', -1))
        batch_size = int(data.get('batch_size', 1))
        
        # Convert base64 to images
        image = base64_to_image(image_base64)
        mask = base64_to_image(mask_base64).convert('L')
        
        # Set seed if provided
        if seed >= 0:
            generator = torch.Generator(device=device).manual_seed(seed)
        else:
            generator = None
        
        print(f'🎨 Generating inpaint: "{prompt}"')
        
        # Generate images
        result = pipe_inpaint(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=image,
            mask_image=mask,
            num_inference_steps=steps,
            guidance_scale=cfg_scale,
            strength=strength,
            num_images_per_prompt=batch_size,
            generator=generator
        )
        
        # Convert images to base64
        images = [image_to_base64(img) for img in result.images]
        
        is_generating = False
        return jsonify({"success": True, "images": images}), 200
    
    except Exception as e:
        is_generating = False
        print(f'❌ Error in inpaint: {e}')
        return jsonify({"error": str(e)}), 500

@app.route('/api/progress', methods=['GET'])
def progress():
    """Get generation progress"""
    return jsonify({"is_generating": is_generating}), 200

@app.route('/api/interrupt', methods=['POST'])
def interrupt():
    """Interrupt generation"""
    global is_generating
    is_generating = False
    return jsonify({"success": True}), 200

print('\n✅ Flask app configured with endpoints:')
print('  • GET  /api/health')
print('  • POST /api/txt2img')
print('  • POST /api/img2img')
print('  • POST /api/inpaint')
print('  • GET  /api/progress')
print('  • POST /api/interrupt')

## Step 5: Start Cloudflared Tunnel (HTTPS)

In [ ]:
import subprocess
import time
import threading

# Start cloudflared in background
cloudflared_process = None

def start_cloudflared():
    """Start cloudflared tunnel"""
    global cloudflared_process
    try:
        cloudflared_process = subprocess.Popen(
            ['cloudflared', 'tunnel', '--url', 'http://localhost:5000'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        # Read output to get tunnel URL
        for line in cloudflared_process.stdout:
            if 'https://' in line:
                tunnel_url = line.split('https://')[1].split()[0]
                print(f'\n🌐 Cloudflared tunnel URL: https://{tunnel_url}')
                print('\n📋 Use this URL in your frontend!')
                break
    except Exception as e:
        print(f'❌ Error starting cloudflared: {e}')

print('Starting cloudflared tunnel...')
cloudflared_thread = threading.Thread(target=start_cloudflared, daemon=True)
cloudflared_thread.start()
time.sleep(2)
print('✅ Cloudflared starting...')

## Step 6: Start Flask Server

In [ ]:
# Start Flask server
def run_flask():
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

print('\n🚀 Flask server started on http://localhost:5000')
print('\n✅ Backend is ready!')
print('\nNext steps:')
print('1. Copy the cloudflared tunnel URL from above')
print('2. Go to your frontend app')
print('3. Paste the URL in #settings tab')
print('4. Click "Test Connection"')
print('5. Start generating images!')

# Keep the notebook running
import time
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print('\n\n⏸️  Server stopped')

## Step 7: Test the API (Optional)
Run this cell to test if the server is working correctly.

In [ ]:
import requests
import json

# Test health endpoint
try:
    response = requests.get('http://localhost:5000/api/health', timeout=5)
    if response.status_code == 200:
        print('✅ API is responding')
        print(json.dumps(response.json(), indent=2))
    else:
        print('❌ API returned error:', response.status_code)
except Exception as e:
    print(f'❌ Cannot connect to API: {e}')
    print('Make sure Step 6 is running')